# C4.5 Decision Tree

# Introduction

In this iPython notebook, we will implement a C4.5 Decision Tree algorithm for machine learning from the scratch.  This is my own Python implementation that utilizes Pandas DataFrame as input data.  The popular Python Scikit-learn decision tree requires Numpy as input that is pure numeric.  Scikit-learn users are required to convert categorical input features to numeric, say color red as 0, blue as 1, green as 2, and the library treats all features the same as continuous numbers.  But I think there is no superior of one color over the others.  Therefore, I implement my own Python version of decision tree that does not require any preprocessing for both numeric and categorical input features, and would process both types of features differently in the internal.

In [3]:
import pandas as pd
import numpy as np

# A structure to store information of a node in a tree 
class Tree:
    # return prediction if the node is a leaf
    prediction = None
    # feature for splitting at a node
    feature = None
    # class or threshold of the feature for splitting
    splitPoint = None
    # a point to left tree node or leaf 
    left = None
    # a point to right tree node or leaf 
    right = None
    # store entropy for analysis
    entropy = None
    # size of the node or leaft
    nCount  = None
    # distribution of target values at the node, e.g. [2 8]
    distribution = None

# Entropy and Information Gain

In short, the entropy $H$ measures the impurity of our target variable $y$, which values are in $k$ classes (boolean counts as 2):

$$ H(y)=-\sum _{ k }P(y=k){ log }_{ 2 }P(y=k) $$
Our algorithm builds decision tree with greedy approach, which, at each node, tries to find a split with maximum information gain on our target.  Suppose we have a target $y$ in size $m$ that is going to split into two subsets ${ y }_{ left }$ and ${ y }_{ right }$, with size ${ m }_{ left }$ and ${ m }_{ right }$ respectively, the information gain is defined as:

$$IG = H(y)-\left[ \frac { { m }_{ left } }{ m } H({ y }_{ left })+\frac { { m }_{ right } }{ m } H({ y }_{ right }) \right] $$

In [4]:
# entropy funciton to support multiple classes
# y takes Pandas Series or 1D Numpy array
def entropy(y):
    dummy, count = np.unique(y.values, return_counts=True)
    p = 1.*count/len(y)
    h = sum(-p*np.log2(p))
    return h

In [5]:
def informationGain(y, y_left, y_right):
    H_mother = entropy(y)
    H_left = entropy(y_left)
    H_right = entropy(y_right)
    H_childern = 1.*(len(y_left) * H_left + len(y_right) * H_right) / len(y)
    return H_mother - H_childern

# Find All Possible Split Points

In [6]:
# test if an input feature is categorical or not based on its data type
# input data X can be Pandas DataFrame, or a row in DataFrame
def isCategorical(X, feature):
    dtype = type(X[feature])
    if dtype == pd.core.series.Series:
        dtype = X[feature].dtype

    return (dtype==str or dtype==object)

In [7]:
# find all possible split points for a feature of both categorical or numeric
def allSplitPoints(X, y, feature):
    # if catgegorical, just returns all unique classes
    if isCategorical(X, feature):
        return X[feature].unique()
    
    # if numeric, sort on input feature, return thresholds where change of classes on y occurs 
    thresholds = []
    view = pd.concat([X[feature], y], axis=1).copy()
    view.sort_values(feature, inplace=True)
    iterator = view.iterrows()
    last_row = iterator.next()[1]
    for dummy, row in (view.iterrows()):
        if row[y.name] != last_row[y.name]:
            thresholds.append((row[feature]+last_row[feature])/2.)
        last_row = row
    return thresholds

# Find the Best Feature to Split

In [8]:
# common function for all feature values comparision in the package
def compare(X, feature, splitPoint):
    # if feature is categorical, test if it falls in the splitPoint as class
    if isCategorical(X, feature):
        return X[feature] == splitPoint
    
    # if features is numeric, test if it's within the splitPoint as threshold
    return X[feature] <= splitPoint

In [9]:
# find the best split point for a feature based on information gain
def findBestSplitPoint(X, y, feature):
    best_ig = 0
    best_splitPoint = None
    
    # loop for all possible split points of a feature
    for splitPoint in allSplitPoints(X, y, feature):
        y_left = y.loc[compare(X,feature,splitPoint)]
        y_right = y.loc[np.logical_not(compare(X,feature,splitPoint))]
        ig = informationGain(y, y_left, y_right)
        # test if the current ig is better than previous, beware rounding
        if ig - best_ig > 0.0001:
            best_ig = ig
            best_splitPoint = splitPoint

    # return None, if no information gain at all
    return (best_splitPoint, best_ig)

In [10]:
# find the best feature to split at a node
# randomFeature is reserved for random forest
def findBestFeature(X, y, randomFeatrue=False):
    best_feature = None
    best_splitPoint = None
    best_ig = 0
    
    # for random forest, randomly pick sqrt(n) features to test
    if randomFeatrue:
        n = int(len(X.columns)**0.5)
        columns = np.random.choice(X.columns, n, replace=False)
    else:
        # otherwise, test all features
        columns = X.columns

    for feature in columns:
        # for each feature, find it's best split point and ig
        splitPoint, ig = findBestSplitPoint(X, y, feature)
        # test if the current ig is better than previous, beware rounding
        if ig - best_ig > 0.0001:
            best_ig = ig
            best_feature = feature
            best_splitPoint = splitPoint

    # return None if no information gain at all
    return (best_feature, best_splitPoint)

# Split Data and Build Tree Recursively

Computer scientists love recursion, it's fun to implement recursion for decision tree.  Our C4.5 decision tree implements the following stopping criteria:

* all targets are in the same class
* cannot find splits with positive information gain
* tree depth grows to maximum limit
* leaf size meets minimum size limit

In [11]:
# split dataset (X,y) given a feature and split pont to split
def splitData(X, y, feature, splitPoint):
    # the splitted dataset should be returned as reference, for performance consideration
    # beware the Pandas characteristic, always use ".loc[]"
    X_left = X.loc[compare(X,feature,splitPoint)]
    y_left = y.loc[compare(X,feature,splitPoint)]
    X_right = X.loc[np.logical_not(compare(X,feature,splitPoint))]
    y_right = y.loc[np.logical_not(compare(X,feature,splitPoint))]
    return (X_left, y_left, X_right, y_right)

In [12]:
from scipy.stats import mode

def voteMajority(y):
    # beware the weird characteristic of Pandas mode function
    # use Scipy instead
    return mode(y)[0][0]

In [13]:
# build tree recursively, return the node built
# X takes Pandas DataFrame, elements can be int, float or string
# y takes Pandas Series, elements can be int, float or string
# depth was used internally, can be ignored
# maxDepth is a model parameter to control the maximum depth of the tree
# minLeafSize is another model parameter, which the leaf size to prevent overfitting
# randomFeature is reserved for random forest, when on, random features are selected to split
def buildTree(X, y, depth=1, maxDepth=50, minLeafSize=5, randomFeatrue=False):
#    print "depth #%d" % depth    
    tree = Tree()
    tree.entropy = entropy(y)
    tree.ncount = y.count()
    tree.distribution = y.value_counts(sort=False).values

    # stop building when node size decreased to minimum leaf size limit
    if len(y)<=minLeafSize:
        tree.prediction = voteMajority(y)
        return tree

    # stop building when all target values are the same
    if y.max()==y.min():
        tree.prediction = voteMajority(y)
        return tree

    # stop building when the node depth reaches the maxmium tree depth limit
    if depth==maxDepth:
        tree.prediction = voteMajority(y)
        return tree

    # try to find the best feature and split point at this node
    feature, splitPoint = findBestFeature(X, y, randomFeatrue)
    
    # stop building when no information gain for any splits
    if feature==None:
        tree.prediction = voteMajority(y)
        return tree

    tree.feature = feature
    tree.splitPoint = splitPoint
    
    # split the node dataset (X,y) into two, and call recursively
    (X_left, y_left, X_right, y_right) = splitData(X, y, feature, splitPoint)
    depth = depth + 1
    tree.left=buildTree(X_left, y_left, depth, 
                        maxDepth=maxDepth, minLeafSize=minLeafSize, 
                        randomFeatrue=randomFeatrue)
    tree.right=buildTree(X_right, y_right, depth, 
                        maxDepth=maxDepth, minLeafSize=minLeafSize, 
                        randomFeatrue=randomFeatrue)
    
    # return the tree node built
    return tree

## Prediction

In [14]:
# return the prediction for an instance of a built tree
# row takes a row from a Pandas DataFrame like X
# tree is a built tree returned from the buildTree function
def predictTree(row, tree):
    if tree.prediction != None:
        return tree.prediction

    if compare(row, tree.feature, tree.splitPoint):
        return predictTree(row, tree.left)
    
    return predictTree(row, tree.right)

In [15]:
# score a tree model, returns accuracy in percentage
# X is input that takes Pandas DataFrame
# y is target that takes Pandas Series
def scoreTree(X, y, tree):
    p = X.apply(lambda x: predictTree(x, tree), axis=1)
    return np.mean(p==y*1.)*100.

## Draw Tree

We use Graphviz to visualize the built decision tree in this package.  Follow [this URL](  https://bobswift.atlassian.net/wiki/display/GVIZ/How+to+install+Graphviz+software) to install the Gaphviz software in your machine.  
Run `pip install graphviz` under the Anaconda environment to install the Python Graphviz library.  
You should be able to display the trained decision tree on iPython Notebook.

In [16]:
from graphviz import Digraph
from uuid import uuid4

def drawGraph(graph, tree):
    node_id = uuid4().hex
    if tree.prediction != None:        
        graph.node(node_id, shape="box", 
                 label="%s\nentropy = %.4f\nsampels = %d\ny %s" 
                 % (tree.prediction, tree.entropy, tree.ncount, 
                    tree.distribution))
        return node_id
    
    graph.node(node_id, shape="box", 
             label="%s | %s\nentropy = %.4f\nsamples = %d\ny %s" 
             % (tree.feature, tree.splitPoint, tree.entropy, tree.ncount, 
                tree.distribution))
    left_id = drawGraph(graph, tree.left)
    graph.edge(node_id, left_id, label="left")
    right_id = drawGraph(graph, tree.right)    
    graph.edge(node_id, right_id, label="right")
    return node_id

## Kaggle Titanic Demo

We are going to test our decision tree package with the dataset from the Titanic data science competition on Kaggle.  For more information, please visit the competition [home page](https://www.kaggle.com/c/titanic).  Here is the data.
<pre>
VARIABLE DESCRIPTIONS:
survival        Survival (0 = No; 1 = Yes)
pclass          Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
</pre>

We first perform some feature engineering to fill missing data and create some new features.
1. fill the missing age with the mdeian
2. fill the missing embarked location with the mode
3. create the `hasCabin` feature that indicates whether a passenger has cabin
4. create the `familySize` feature which adds up `SibSp` and `Parch`

As our training pool is small (around 900 samples), and the no. of features is small, we use the following model parameters to train a tree to prevent overfitting:
1. maximum tree depth: 4
2. minimum leaf size: 20

Here is the resulting tree:
<img src="./titanic_lv4.png"/>

In [17]:
def prepareTitanic(filename):
    df = pd.read_csv(filename)
    df["Age"] = df["Age"].fillna(df["Age"].median())
    df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode().iloc[0])
    df["hasCabin"] = df["Cabin"].apply(lambda x: "true" if pd.notnull(x) else "false")
    df["familySize"] = df["SibSp"] + df["Parch"]
    
    X = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", 
            "hasCabin", "familySize"]]
    y = df["Survived"]
    return (X, y)

In [21]:
import time

# load the traning data
X, y = prepareTitanic("train.csv")
start_time = time.time()
# train a tree
tree = buildTree(X, y, maxDepth=4, minLeafSize=20)
elapsed_time = time.time() - start_time
print ("Time elapsed: %.2f" % elapsed_time)
# score the accuracy
accuracy = scoreTree(X, y, tree)
print ("Accuracy %.2f"%accuracy)

graph = Digraph(comment="Titanic Survival Descision Tree")
drawGraph(graph, tree)

# uncomment to draw your tree
#graph

Time elapsed: 13.44
Accuracy 81.93


'64ec8d2392e14c1e84b806210fcc5648'

## Final Thoughts

I tried to generate a submission for the Kaggle Titan competition on the testing data with this decision tree model, and it scored 78.947%. If we grow the tree into more depths, say maximum depth 5, the accuracy for training set data would be higher. But in contrast, the accuracy for unseen testing data (77.512%) would be lower. In machine learning language, it is called overfitting.  Although our decision tree model result was better than simply assuming only all female passengers survive (76.555%), the generalization ability for a single decision tree model is limited.  In the next iPython Notebook, we will see how to improve the prediction performance by building multiple trees, i.e. random forest.